# Inspect and summarize SLURM benchmarks

## Overview

## Load, merge and compare benchmark summary `csv`s

We need a function to compare our SLURM summary output files and highlight differences in a table.

In [1]:
library(dplyr)
library(tidyr)

read_and_summarize_differences <- function(directory) {
  # List all CSV files in the directory
  files <- list.files(directory, pattern = "\\.csv$", full.names = TRUE)
  files <- files[!grepl("1000000", files)]
  
  # Read each file and store in a list
  data_list <- lapply(files, function(file) {
    data <- read.csv(file)
    # Add a column to identify the source file
    data$Source <- basename(file)
    return(data)
  })
  
  # Combine all data frames into one
  combined_data <- bind_rows(data_list)
  
  # Spread the data to wide format for comparison
  wide_data <- spread(combined_data, key = Parameter, value = Value)
  
  # Identify rows with at least one difference in values
  differing_rows <- wide_data %>%
    summarise(across(-Source, ~n_distinct(.) > 1)) %>%
    select(which(colSums(.) > 0)) %>%
    names()
  
  # Filter the combined data to include only differing parameters
  summary_data <- combined_data %>%
    filter(Parameter %in% differing_rows) %>%
    select(Parameter, Source, Value)
  
  # Pivot wider for a clearer summary table, if necessary
  summary_table <- summary_data %>%
    pivot_wider(names_from = Source, values_from = Value)
  
  return(summary_table)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
getwd()

[1] "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts"

In [3]:
directory <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/output_chr1_a2_dlpfc/"
summary_table <- read_and_summarize_differences(directory)
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240421-204642-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240421-205043-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240421-205444-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240421-205845-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240421-210647-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1128982-1148981-dynamic-1corestotal-allcorepera-20240421-211048-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1188982-1208981-dynamic-1corestotal-allcorepera-20240421-212252-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1228982-1248981-dynamic-1corestotal-allcorepera-20240421-213054-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1248982-1268981-dynamic-1corestotal-allcorepera-20240421-213455-summary.csv,⋯,libd_chr1-chr1_EA-libd_chr1-chr1_EA-588982-608981-dynamic-1corestotal-allcorepera-20240420-090346-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-68982-88981-dynamic-1corestotal-allcorepera-20240420-074505-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-748982-768981-dynamic-1corestotal-allcorepera-20240420-092755-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-848982-868981-dynamic-1corestotal-allcorepera-20240420-094301-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-868982-888981-dynamic-1corestotal-allcorepera-20240420-094602-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-88982-108981-dynamic-1corestotal-allcorepera-20240420-074806-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-928982-948981-dynamic-1corestotal-allcorepera-20240420-095505-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-948982-968981-dynamic-1corestotal-allcorepera-20240420-095806-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-968982-988981-dynamic-1corestotal-allcorepera-20240420-100107-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-988982-1008981-dynamic-1corestotal-allcorepera-20240420-100408-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
methylation_data_path,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,⋯,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda
tag,libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240421-204642,libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240421-205043,libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240421-205444,libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240421-205845,libd_chr1-chr1_AA

## Compute core-hours per test

In [4]:
windows <- c(10000)
n_windows <- length(windows)
alphas <- seq(0.5) # left out 0 on first run
n_alphas <- length(alphas)
#chunks <- 10000
#n_tests <- n_windows * n_alphas * chunks
#n_tests

In [5]:
alphas

[1] 1

In [6]:
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240421-204642-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240421-205043-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240421-205444-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240421-205845-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240421-210647-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1128982-1148981-dynamic-1corestotal-allcorepera-20240421-211048-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1188982-1208981-dynamic-1corestotal-allcorepera-20240421-212252-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1228982-1248981-dynamic-1corestotal-allcorepera-20240421-213054-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1248982-1268981-dynamic-1corestotal-allcorepera-20240421-213455-summary.csv,⋯,libd_chr1-chr1_EA-libd_chr1-chr1_EA-588982-608981-dynamic-1corestotal-allcorepera-20240420-090346-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-68982-88981-dynamic-1corestotal-allcorepera-20240420-074505-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-748982-768981-dynamic-1corestotal-allcorepera-20240420-092755-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-848982-868981-dynamic-1corestotal-allcorepera-20240420-094301-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-868982-888981-dynamic-1corestotal-allcorepera-20240420-094602-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-88982-108981-dynamic-1corestotal-allcorepera-20240420-074806-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-928982-948981-dynamic-1corestotal-allcorepera-20240420-095505-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-948982-968981-dynamic-1corestotal-allcorepera-20240420-095806-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-968982-988981-dynamic-1corestotal-allcorepera-20240420-100107-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-988982-1008981-dynamic-1corestotal-allcorepera-20240420-100408-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
methylation_data_path,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,⋯,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_EA.rda
tag,libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240421-204642,libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240421-205043,libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240421-205444,libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240421-205845,libd_chr1-chr1_AA

In [7]:
summary_table <- as.data.frame(t(summary_table))
colnames(summary_table) <- summary_table[1, ]
summary_table <- summary_table[-1, ]

In [8]:
#summary_table$number_cores <- as.numeric(as.character(summary_table$num_cores))
summary_table$number_cores <- 1

In [9]:
summary_table$runtime_seconds <- sapply(summary_table$runtime, function(x) {
  parts <- strsplit(x, ":")[[1]]
  as.numeric(parts[1]) * 3600 + as.numeric(parts[2]) * 60 + as.numeric(parts[3])
})

In [10]:
summary_table$runtime_hours <- summary_table$runtime_seconds/3600

In [11]:
min(summary_table$chunk2)

Warning message in min(summary_table$chunk2):
“no non-missing arguments to min; returning Inf”


[1] Inf

In [12]:
chunk_df <- data.frame(stringr::str_split_fixed(summary_table$tag, "-", 5)[, 3:4])

In [13]:
colnames(chunk_df) <- c("chunk1", "chunk2")

In [14]:
dim(summary_table)

[1] 121  14

In [15]:
dim(chunk_df)

[1] 121   2

In [16]:
summary_table <- cbind(summary_table, chunk_df)

In [17]:
summary_table$chunk1 <- as.numeric(as.character(summary_table$chunk1))
summary_table$chunk2 <- as.numeric(as.character(summary_table$chunk2))

In [18]:
summary_table$runtime_core_hours <- summary_table$runtime_hours * summary_table$number_cores
summary_table$runtime_core_seconds <- summary_table$runtime_seconds * summary_table$number_cores
summary_table$core_hours_per_test <- summary_table$runtime_core_hours / (summary_table$chunk2 - summary_table$chunk1 + 1)
summary_table$core_seconds_per_test <- summary_table$runtime_core_seconds / (summary_table$chunk2 - summary_table$chunk1 + 1)

In [19]:
summary_table

,methylation_data_path,tag,methInput_rds_path,cov,time_started,time_finished,runtime,type_CPU,amount_RAM,physical_cores,scaffold_ID,number_cores,runtime_seconds,runtime_hours,chunk1,chunk2,runtime_core_hours,runtime_core_seconds,core_hours_per_test,core_seconds_per_test
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240421-204642-summary.csv,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240421-204642,/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA_1008982-1028981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_dlpfc.csv,2024-04-21 20:47:00,2024-04-22 10:44:17,13:57:17,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.245873626321554,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240421-204642,1,50237,13.95472222,1008982,1028981,13.95472222,50237,6.977361e-04,2.51185
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240421-205043-summary.csv,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240421-205043,/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA_1028982-1048981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_dlpfc.csv,2024-04-21 20:51:01,2024-04-22 07:21:05,10:30:04,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.122832637280226,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240421-205043,1,37804,10.50111111,1028982,1048981,10.50111111,37804,5.250556e-04,1.89020
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240421-205444-summary.csv,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240421-205444,/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA_1048982-1068981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_dlpfc.csv,2024-04-21 20:55:01,2024-04-22 10:35:51,13:40:49,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.245873626321554,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240421-205444,1,49249,13.68027778,1048982,1068981,13.68027778,49249,6.840139e-04,2.46245
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240421-205845-summary.csv,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240421-205845,/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA_1068982-1088981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_dlpfc.csv,2024-04-21 20:59:01,2024-04-22 09:39:29,12:40:28,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.245873626321554,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240421-205845,1,45628,12.67444444,1068982,1088981,12.67444444,45628,6.337222e-04,2.28140
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240421-210647-summary.csv,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/dlpfc/out/chr1_AA.rda,libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240421-210647,/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA_1108982-1128981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_dlpfc.csv,2024-04-21 21:07:02,2024-04-22 07:32:56,10:25:54,Intel(R) Xeon(R) CPU E5-2650 v4 @ 2.20GHz,0.122523229569197,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240421-210647,1,37554,10.43166667,1108982,1128981,10.43166667,37554,5.215833e-04,1.87770
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1128982-1148981-dynamic-1corestotal-allcorepera-20240421-211048-summary.csv,/dcs04/

In [20]:
#summary_table[which.min(summary_table$core_hours_per_test), ]

In [21]:
data.table::fwrite(summary_table, "05-OUT_SLURM_benchmarks_a5_chr1_dlpfc.csv")

## How long for whole dataset?

***This is for benchmarks with 500kb windows***

In [22]:
#summary_table <- summary_table[which(summary_table$num_cores == 1), ]

In [23]:
n_sites <- 26000000

In [24]:
# Core hours for single run over methylation dataset (single brain region, single population dataset)

n_sites*mean(summary_table$core_hours_per_test)

[1] 10410.2

In [25]:
# Core hours for single run over methylation dataset (single brain region, single population dataset)

n_sites*mean(summary_table$core_hours_per_test)

[1] 10410.2

In [26]:
# Core hours for 3 x 3 brain regions x population datasets

brain_regions <- 3
populations <- 3
n_sites*mean(summary_table$core_hours_per_test)*brain_regions*populations

[1] 93691.78